In [1]:
%load_ext autoreload
%autoreload 2

import os, sys

sys.path.insert(0, "src")

from pandas import DataFrame
from analytics.metrics.metric import MetricCalculator
from analytics.metrics.time import TimeMetrics
from spec.enums import MainTableColumns as Cols
from spec.spec_definition import PS2Versions

In [ ]:
from database.config import PS2DataConfig

spec = PS2Versions.v1_0.load()

data_dir = "sample_data/cwo/s19"
config = os.path.join(data_dir, "data_config.yaml")

data_config = PS2DataConfig.from_yaml(config, spec)
data_config.csv_path


{'root_path': '.', 'csv_path': 'MainTable.csv', 'optimize_codestate_ids': True, 'metadata': {'Version': '1.0', 'IsEventOrderingConsistent': True, 'EventOrderScope': 'Global', 'CodeStateRepresentation': 'Table'}}


'MainTable.csv'

In [5]:
from database.writer.db_writer_factory import IOFactory
factory = IOFactory.create_factory(data_config)
with factory.create_reader() as reader:
    reader = reader

Checking version: 1.0 against 1.0


AttributeError: 'IOContext' object has no attribute 'conn'

In [ ]:

original_df = DataFrame()

time_metrics = TimeMetrics()

results = time_metrics.calculate(original_df.groupby([Cols.SubjectID, Cols.ProblemID]))